In [1]:
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cmocean as cmo
import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy as np
import copy
import sys,os

sys.path.append('../src')
import utils as ut

In [2]:
#General input params

run  = 'tot2'  # New run to be compared to control
jmax = 55      # Maximum j index for northern latitudinal extent
olev = 36      # Depth index, 34 = 300m, 36 = 370m

In [3]:
def get_fnames(run):
    fnames = sorted(glob.glob(f'../data/ecefiles/{run}/{run}*T.nc'))
    if run == 'n011':
        fnames = fnames[38:] #Omit the first 100 years which is spin up
    #    rrun = 'ctrl'
    #else:
    #    rrun = run

    nmonths = 0
    for fname in fnames:
        ds = xr.open_dataset(fname)
        nmonths += len(ds.time_counter)
        ds.close()
    nyears = int(np.floor(nmonths/12))
    print(run,nmonths, 'months =',nyears,'y',int(nmonths-12*nyears),'m')
    return fnames,nmonths

In [4]:
ds = xr.open_dataset(f'../data/basinmask.nc').isel(y=slice(0,jmax),basin=slice(0,5))
lon = ds.lon
lat = ds.lat
mask = ds.mask
basin = ds.basin.values
ds.close()

mask = np.nansum(mask,axis=1)

In [5]:
runs = np.append([run],'n011')

ds = xr.open_dataset(f'../data/temperature_mon_ctrl.nc').isel(basin=slice(0,5))
tempc = ds.temp
ds.close()

ds = xr.open_dataset(f'../data/temperature_mon_{run}.nc').isel(basin=slice(0,5))
time = ds.time.values/12
tempr = ds.temp
ds.close()

dtemp = tempr-tempc[:len(time)]
nmonths = len(time)
nyears = int(np.floor(nmonths/12))
print(nmonths,nyears,nmonths-12*nyears)

['tot2' 'n011']
403 33 7


In [6]:
var = {}
for rn in runs:
    fnames,nmon = get_fnames(rn)
    if rn==run:
        assert nmon==nmonths,'Need to run Read_ece_ts_oceantemp'
    var[rn] = np.zeros((nmonths,lon.shape[0],lon.shape[1]))
    c = -1
    for fname in fnames:
        ds = xr.open_dataset(fname).isel(y=slice(0,jmax))
        for t,tt in enumerate(ds['time_counter'].values):
            c+=1
            if c>=nmonths:
                break
            var[rn][c,:,:] = ds['thetao'].isel(olevel=olev,time_counter=t)
        ds.close()
        if c>=nmonths:
            break
        

tot2 403 months = 33 y 7 m
n011 2400 months = 200 y 0 m


In [34]:
cmap = 'cmo.balance'
vmin = -1
vmax = 1

mpl.rcParams['figure.figsize'] = (10,5)
mpl.rcParams['figure.subplot.wspace'] = .15
mpl.rcParams['figure.subplot.hspace'] = .01
mpl.rcParams['figure.subplot.left'] = .01
mpl.rcParams['figure.subplot.right'] = .99
mpl.rcParams['figure.subplot.bottom'] = .1
mpl.rcParams['figure.subplot.top'] = .9

mnames = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [40]:
prefix = f'substemp_{run}_mon'

framerate = nmonths/60

#for m in range(nmonths-2,nmonths):
for m in range(nmonths):
    fig = plt.figure()
        
    ax = fig.add_subplot(121,projection=ccrs.SouthPolarStereo())
    ax.set_extent([-180, 180, -80, -59], crs=ccrs.PlateCarree())
    im = ax.scatter(lon,lat,5,c=var['tot2'][m,:,:]-var['n011'][m,:,:],cmap=cmap,vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
    #im = ax.pcolormesh(lon[:,1:-1],lat[:,1:-1],var['tot2'][m,:,1:-1]-var['n011'][m,:,1:-1],cmap=cmap,vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
    #im = ax.pcolormesh(lon,lat,var['tot2'][m,:,:]-var['n011'][m,:,:],cmap=cmap,vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())

    ax.add_feature(cartopy.feature.LAND.with_scale('10m'),facecolor='.7')
    ax.coastlines(linewidth=0.4, zorder=2, resolution='10m')
        
    axins = inset_axes(ax,width="5%",height="100%",loc='center right',borderpad=-2)
    cb = fig.colorbar(im, cax=axins)
    
    ax.set_title('Temperature anomaly at 370m depth [degC]')
    ax.set_axis_off()
    
    ax = fig.add_subplot(122)
    ax.axhline(0,0,1,color='.5',ls=':')
    for b,bas in enumerate(basin):
        ax.plot(time[:m+1],dtemp.sel(basin=bas)[:m+1],c=ut.bcol[bas])
        ax.scatter(time[m],dtemp.sel(basin=bas)[m],40,c=ut.bcol[bas])
    ax.set_xlim([0,np.ceil(nmonths/12)])
    ax.set_ylim([vmin,vmax]) 
    ax.set_yticklabels([])
    ax.set_title('Subsurface ocean temperature anomaly [degC]')
    ax.set_xlabel('Years')

    axins = inset_axes(ax,width="30%",height="30%",loc='lower left',borderpad=0,axes_class=cartopy.mpl.geoaxes.GeoAxes,axes_kwargs=dict(map_projection=cartopy.crs.SouthPolarStereo()))
    axins.set_extent([-180, 180, -80, -60], crs=ccrs.PlateCarree())
    for b,bas in enumerate(basin):
        axins.scatter(lon,lat,5*mask[b,:,:],c=ut.bcol[bas],lw=0,transform=ccrs.PlateCarree())

    axins.add_feature(cartopy.feature.LAND.with_scale('10m'),facecolor='.7')
    axins.coastlines(linewidth=0.4, zorder=2, resolution='10m')
    axins.set_axis_off()
    
    y2d = int(np.floor(m/12))
    m2d = int(m-12*y2d)
    fig.suptitle(f'{mnames[m2d]} {y2d:03.0f} of {nyears:03.0f}')
    
    savename = f'../videos/{prefix}_{m:04.0f}.png'
    plt.savefig(savename,dpi=150,facecolor='w',transparent=False)
    print('Saved',m,'of',nmonths,'months',end='\r')
    plt.close()
    
os.system(f'ffmpeg -y -r {framerate} -f image2 -s 1920x1080 -i ../videos/{prefix}_%04d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p ../videos/{prefix}.mp4')
os.system(f'rm -r ../videos/{prefix}*.png')

ffmpeg version 4.3.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-D_FORTIFY_SOURCE=2 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld ' --extra-cflags=' -I/usr/include/rav1e' --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libvo-amrwbenc --enable-version3 --enable-bzlib --disable-crystalhd --enable-fontconfig --enable-frei0r --enable-gcrypt --enabl

0